Based on:

https://www.dominodatalab.com/blog/named-entity-recognition-ner-challenges-and-model

https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

# Install packages

In [6]:
!pip install plot_keras_history
!pip install keras
!pip uninstall tensorflow -y
!pip install tensorflow
!pip install pyconll
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install conllu
!pip install flair

You should consider upgrading via the '/home/digitalhub/.pyenv/versions/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/digitalhub/.pyenv/versions/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Successfully uninstalled tensorflow-2.11.0
  Using cached h5py-2.10.0-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Using cached importlib_metadata-5.1.0-py3-none-any.whl (21 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
  

# Import packages

In [3]:
import pickle
import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyconll
import tensorflow as tf
import flair

from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from keras_contrib.utils import save_load_utils

from keras import layers
from keras import optimizers

from keras.models import Model
from keras.layers import Input

from keras_contrib.layers import CRF
from keras_contrib import losses
from keras_contrib import metrics

from typing import List
from flair.data import Corpus
from flair.datasets import ColumnCorpus, CONLL_03
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.embeddings import WordEmbeddings, TokenEmbeddings, FlairEmbeddings, PooledFlairEmbeddings, StackedEmbeddings

2022-12-12 13:51:20.206422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 13:51:20.206435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Using TensorFlow backend.
Using TensorFlow backend.
Using TensorFlow backend.


ImportError: cannot import name 'transpose_shape' from 'keras.utils.generic_utils' (/home/digitalhub/.pyenv/versions/3.8.12/lib/python3.8/site-packages/keras/utils/generic_utils.py)

In [5]:
flair.__version__

'0.11.3'

# Pre-trained library

In [16]:
# Create some test sentences
sentences = [
            'My name is Clara, I live in Singapore, I work for DSTA.',
            'Missiles and other explosives or ammonium nitrate are dangerous.',
            'Ammonium nitrate was also used in the IRA\'s bombing of London\'s Canary Wharf in 1996.',
            'These have included such naval craft as the ship of the line, battleship, submarine, and aircraft carrier; aircraft such as the fighter, bomber, and helicopter; and ground vehicles such as the chariot and tank.',
            'George Washington went to Washington'
            ]

# load the NER tagger
tagger = SequenceTagger.load('ner')

for i in sentences: 

    sentence = Sentence(i)

    # run NER over sentence
    tagger.predict(sentence)

    # print sentence
    print('SENTENCE: ' + i)
    print('----------------------------------------------------------------')

    # print predicted NER spans
    print('The following NER tags are found:')
    # iterate over entities and print
    for entity in sentence.get_spans('ner'):
        print(entity)
    
    print('\n')

/home/digitalhub/.pyenv/versions/3.8.12/lib/python3.8/site-packages/huggingface_hub/file_download.py:592: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-12-12 11:24:30,717 loading file /home/digitalhub/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-12-12 11:24:32,061 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
SENTENCE: My name is Clara, I live in Singapore, I work for DSTA.
----------------------------------------------------------------
The following NER tags are found:
Span[3:4]: "Clara" → PER (0.9991)
Span[8:9]: "Singapore" → LOC (0.9993)
Span[13:14]: "DSTA" → ORG (0.9981)


SENTENCE: Missiles and other explosives or ammonium nitrate are dangerous.
----------------------------------------------------------------
The following NER tags are found:


SENTENCE: Ammonium nitrate was also used in the IRA's bombing of London's Canary Wharf in 1996.
----------------------------

# Stacked (Word + Flair forward + backward)

## On domain-specific (re3d) data

In [3]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='val.conll')

2022-12-08 10:11:33,761 Reading data from entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits
2022-12-08 10:11:33,763 Train: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/train.conll
2022-12-08 10:11:33,763 Dev: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/val.conll
2022-12-08 10:11:33,764 Test: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/test.conll


In [4]:
print(corpus.train[0].to_tagged_string('ner'))
print(len(corpus.train))

Sentence: "Operation Steel Curtain ( Arabic : ا ل ح ج ا ب ا ل ف و ل ا ذ ي Al Hejab Elfulathi ) was a military operation executed by coalition forces in early November 2005 to reduce the flow of foreign insurgents crossing the border and joining the Iraqi insurgency ." → ["coalition forces"/Organisation, "early November 2005"/Temporal, "foreign insurgents"/Organisation, "Iraqi"/Nationality]
771


In [5]:
# Define tags to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2022-12-08 10:11:34,192 Computing label dictionary. Progress:


771it [00:00, 46820.65it/s]

2022-12-08 10:11:34,246 Dictionary created for label 'ner' with 11 values: Organisation (seen 1113 times), Location (seen 647 times), Person (seen 415 times), Temporal (seen 179 times), Quantity (seen 151 times), Weapon (seen 81 times), DocumentReference (seen 50 times), MilitaryPlatform (seen 48 times), Nationality (seen 34 times), Money (seen 20 times)


In [6]:
# Embeddings

from typing import List
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

In [7]:
embedding_types = [
    # init standard GloVe embedding
    WordEmbeddings('glove'),

    # init Flair forward and backwards embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)


In [8]:
from flair.models import SequenceTagger

# initialize a BiLSTM-CRF sequence tagger
tagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

from flair.trainers import ModelTrainer

# initialize trainer
trainer = ModelTrainer(tagger, corpus)

# start training
trainer.train('resources/taggers/re3d-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-12-08 10:11:37,419 SequenceTagger predicts: Dictionary with 41 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=

/home/digitalhub/.pyenv/versions/3.8.12/lib/python3.8/site-packages/flair/trainers/trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2022-12-08 10:11:45,608 epoch 1 - iter 2/25 - loss 3.51568870 - samples/sec: 7.96 - lr: 0.100000
2022-12-08 10:11:53,708 epoch 1 - iter 4/25 - loss 2.86969593 - samples/sec: 7.90 - lr: 0.100000
2022-12-08 10:12:00,867 epoch 1 - iter 6/25 - loss 2.60209506 - samples/sec: 8.94 - lr: 0.100000
2022-12-08 10:12:09,591 epoch 1 - iter 8/25 - loss 2.39064374 - samples/sec: 7.34 - lr: 0.100000
2022-12-08 10:12:17,554 epoch 1 - iter 10/25 - loss 2.27275103 - samples/sec: 8.04 - lr: 0.100000
2022-12-08 10:12:25,963 epoch 1 - iter 12/25 - loss 2.16142430 - samples/sec: 7.61 - lr: 0.100000
2022-12-08 10:12:41,867 epoch 1 - iter 14/25 - loss 2.13654410 - samples/sec: 4.02 - lr: 0.100000
2022-12-08 10:12:51,122 epoch 1 - iter 16/25 - loss 2.07022120 - samples/sec: 6.92 - lr: 0.100000
2022-12-08 10:12:59,732 epoch 1 - iter 18/25 - loss 2.03364964 - samples/sec: 7.43 - lr: 0.100000
2022-12-08 10:13:09,244 epoch 1 - iter 20/25 - loss 2.03030997 - samples/sec: 6.73 - lr: 0.100000
2022-12-08 10:13:20,845 

100%|██████████| 4/4 [00:12<00:00,  3.08s/it]

2022-12-08 10:13:42,610 Evaluating as a multi-label problem: False
2022-12-08 10:13:42,620 DEV : loss 1.4683295488357544 - f1-score (micro avg)  0.0
2022-12-08 10:13:42,631 BAD EPOCHS (no improvement): 0
2022-12-08 10:13:42,632 ----------------------------------------------------------------------------------------------------


...

2022-12-08 10:34:41,173 Evaluating as a multi-label problem: False
2022-12-08 10:34:41,182 DEV : loss 0.29154881834983826 - f1-score (micro avg)  0.6742
2022-12-08 10:34:41,193 Epoch    65: reducing learning rate of group 0 to 9.7656e-05.
2022-12-08 10:34:41,194 BAD EPOCHS (no improvement): 4
2022-12-08 10:34:41,195 ----------------------------------------------------------------------------------------------------
2022-12-08 10:34:41,195 ----------------------------------------------------------------------------------------------------
2022-12-08 10:34:41,195 learning rate too small - quitting training!
2022-12-08 10:34:41,196 ----------------------------------------------------------------------------------------------------


2022-12-08 10:34:41,846 ----------------------------------------------------------------------------------------------------
2022-12-08 10:34:41,847 loading file resources/taggers/re3d-ner/best-model.pt
2022-12-08 10:34:42,448 SequenceTagger predicts: Dictionary with 43 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money, <START>, <STOP>


100%|██████████| 3/3 [00:13<00:00,  4.48s/it]

2022-12-08 10:34:56,107 Evaluating as a multi-label problem: False
2022-12-08 10:34:56,113 0.6744	0.7335	0.7027	0.5519
2022-12-08 10:34:56,114 
Results:
- F-score (micro) 0.7027
- F-score (macro) 0.4017
- Accuracy 0.5519

By class:
                   precision    recall  f1-score   support

     Organisation     0.6561    0.7357    0.6936       140
         Location     0.6712    0.7206    0.6950        68
           Person     0.7910    0.8548    0.8217        62
         Temporal     0.9474    0.9474    0.9474        19
         Quantity     0.4545    0.6250    0.5263        16
           Weapon     0.0000    0.0000    0.0000         4
DocumentReference     0.5000    0.2500    0.3333         4
            Money     0.0000    0.0000    0.0000         1
      Nationality     0.0000    0.0000    0.0000         2
 MilitaryPlatform     0.0000    0.0000    0.0000         3

        micro avg     0.6744    0.7335    0.7027       319
        macro avg     0.4020    0.4134    0.4017       319

{'test_score': 0.7027027027027027,
 'dev_score_history': [0.0,
  0.10352941176470588,
  0.2033271719038817,
  0.3082437275985663,
  0.3926940639269407,
  0.5240174672489083,
  0.5384615384615385,
  0.5873715124816447,
  0.5794655414908579,
  0.6298507462686567,
  0.6127527216174184,
  0.6171428571428571,
  0.6490455212922173,
  0.6500000000000001,
  0.6296809986130374,
  0.64756446991404,
  0.6568483063328424,
  0.6334716459197788,
  0.6452554744525547,
  0.6666666666666666,
  0.6273458445040215,
  0.6820276497695852,
  0.6277173913043478,
  0.6785185185185185,
  0.6666666666666667,
  0.663768115942029,
  0.674863387978142,
  0.6805970149253732,
  0.6806136680613668,
  0.6737967914438503,
  0.662937062937063,
  0.6827880512091038,
  0.7075606276747504,
  0.6853932584269663,
  0.6914285714285714,
  0.6749654218533887,
  0.6733238231098431,
  0.6835443037974684,
  0.6751054852320675,
  0.6816901408450704,
  0.6779661016949153,
  0.6797752808988764,
  0.6779184247538679,
  0.6704225352112

In [16]:
from flair.data import Sentence
from flair.models import SequenceTagger

input_sentence = 'My name is Clara, I live in Singapore, I work for DSTA.'
input_sentence = 'Missiles and other explosives or ammonium nitrate are dangerous.'
input_sentence = 'Ammonium nitrate was also used in the IRA\'s bombing of London\'s Canary Wharf in 1996.'
#input_sentence = 'These have included such naval craft as the ship of the line, battleship, submarine, and aircraft carrier; aircraft such as the fighter, bomber, and helicopter; and ground vehicles such as the chariot and tank.'
tagger: SequenceTagger = SequenceTagger.load("resources/taggers/re3d-ner/best-model.pt")
sentence: Sentence = Sentence(input_sentence)
tagger.predict(sentence)
print(sentence.to_tagged_string())

2022-12-08 11:35:14,711 loading file resources/taggers/re3d-ner/best-model.pt
2022-12-08 11:35:15,643 SequenceTagger predicts: Dictionary with 43 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money, <START>, <STOP>
Sentence: "Ammonium nitrate was also used in the IRA 's bombing of London 's Canary Wharf in 1996 ." → ["IRA"/Organisation, "London 's Canary Wharf"/Location, "1996"/Temporal]


## On general (docred) data

In [ ]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'docred_preprocessed'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

downsampled_corpus = corpus.downsample(0.1)

2022-12-09 11:16:58,545 Reading data from docred_preprocessed
2022-12-09 11:16:58,547 Train: docred_preprocessed/train.txt
2022-12-09 11:16:58,549 Dev: docred_preprocessed/dev.txt
2022-12-09 11:16:58,549 Test: docred_preprocessed/test.txt


In [ ]:
print(downsampled_corpus.train[0].to_tagged_string('ner'))
print(len(downsampled_corpus.train))

Sentence: "Del ia ĠBuckley ĠGallagher Ġ( Ġborn Ġ11 ĠMarch Ġ1970 Ġ) Ġis Ġan ĠAmerican Ġjournalist Ġbased Ġin ĠRome Ġwho Ġcurrently Ġserves Ġas Ġthe ĠSenior ĠEditor Ġfor ĠInside Ġthe ĠVatican Ġmagazine Ġ. She Ġformerly Ġserved Ġas ĠCNN ĠâĢ Ļ s ĠFaith Ġand ĠValues ĠCorrespond ent Ġ. Based Ġin ĠNew ĠYork Ġ, ĠGallagher Ġwas Ġa Ġlong Ġtime ĠCNN ĠVatican ĠAnalyst Ġ, ĠVatican ologist Ġ, Ġand Ġreligious Ġjournalist Ġ. Prior Ġto Ġjoining ĠCNN Ġfull Ġtime Ġ, Ġshe Ġlived Ġin ĠRome Ġfor Ġ7 Ġyears Ġ. In ĠRome Ġ, Ġshe Ġwrote Ġa Ġweekly Ġcolumn Ġfor ĠZen it ĠNews ĠAgency Ġand Ġwas Ġa Ġcontributing Ġeditor Ġfor Ġthe Ġmagazine Ġ. The ĠHistory ĠChannel ĠDocument ary Ġ, Ġ" ĠAngels Ġand ĠDemons ĠDec oded Ġ" Ġreleased Ġby ĠA Ġ& ĠE ĠTelevision ĠNetworks Ġprof iled ĠGallagher Ġcommenting Ġon ĠDan ĠBrown Ġ' s Ġbest seller Ġbook Ġwhich Ġwas Ġmade Ġinto Ġa Ġmovie Ġ. Sub sequently Ġ, ĠGallagher Ġmoder ated Ġthe ĠAngels Ġand ĠDemons Ġmovie Ġpress Ġconference Ġfilm Ġdebut Ġin ĠRome Ġon Ġstage Ġwith ĠTom ĠH anks Ġ, 

In [ ]:
# Define tags to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = downsampled_corpus.make_label_dictionary(label_type=tag_type)

2022-12-09 10:56:58,844 Computing label dictionary. Progress:


3008it [00:00, 25790.95it/s]

2022-12-09 10:56:59,000 Dictionary created for label 'ner' with 7 values: LOC (seen 10903 times), PER (seen 5336 times), MISC (seen 4419 times), ORG (seen 4334 times), TIME (seen 3340 times), NUM (seen 8 times)


In [ ]:
# Embeddings

from typing import List
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

In [ ]:
embedding_types = [
    # init standard GloVe embedding
    WordEmbeddings('glove'),

    # init Flair forward and backwards embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)


In [ ]:
from flair.models import SequenceTagger

# initialize a BiLSTM-CRF sequence tagger
tagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

from flair.trainers import ModelTrainer

# initialize trainer
trainer = ModelTrainer(tagger, downsampled_corpus)

# start training
trainer.train('resources/taggers/docred-ner',
              learning_rate=0.1,
              mini_batch_size=64,
              max_epochs=150)

2022-12-09 11:18:12,896 SequenceTagger predicts: Dictionary with 25 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, S-TIME, B-TIME, E-TIME, I-TIME, S-NUM, B-NUM, E-NUM, I-NUM
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked

100%|██████████| 1/1 [00:29<00:00, 29.36s/it]

2022-12-09 11:24:50,576 Evaluating as a multi-label problem: False
2022-12-09 11:24:50,581 DEV : loss 0.9319246411323547 - f1-score (micro avg)  0.0
2022-12-09 11:24:50,600 BAD EPOCHS (no improvement): 0
2022-12-09 11:24:50,601 ----------------------------------------------------------------------------------------------------


2022-12-09 11:25:20,274 epoch 2 - iter 1/5 - loss 0.90134813 - samples/sec: 2.16 - lr: 0.100000
2022-12-09 11:25:48,091 epoch 2 - iter 2/5 - loss 0.93840038 - samples/sec: 2.30 - lr: 0.100000
2022-12-09 11:26:19,322 epoch 2 - iter 3/5 - loss 0.91924218 - samples/sec: 2.05 - lr: 0.100000
2022-12-09 11:26:45,252 epoch 2 - iter 4/5 - loss 0.93913768 - samples/sec: 2.47 - lr: 0.100000
2022-12-09 11:27:10,925 epoch 2 - iter 5/5 - loss 0.93488603 - samples/sec: 2.49 - lr: 0.100000
2022-12-09 11:27:10,926 ----------------------------------------------------------------------------------------------------
2022-12-09 11:27:10,926 EPOCH 2 done: loss 0.9349 - lr 0.100000


100%|██████████| 1/1 [00:01<00:00,  1.64s/it]

...

2022-12-09 11:49:36,713 Evaluating as a multi-label problem: False
2022-12-09 11:49:36,727 DEV : loss 0.7337616086006165 - f1-score (micro avg)  0.0
2022-12-09 11:49:36,749 BAD EPOCHS (no improvement): 0
2022-12-09 11:49:36,750 ----------------------------------------------------------------------------------------------------


2022-12-09 11:50:04,201 ----------------------------------------------------------------------------------------------------
2022-12-09 11:50:04,202 Exiting from training early.
2022-12-09 11:50:04,203 Saving model ...
2022-12-09 11:50:05,187 Done.
2022-12-09 11:50:05,188 ----------------------------------------------------------------------------------------------------
2022-12-09 11:50:05,188 Testing using last state of model ...


100%|██████████| 2/2 [01:00<00:00, 30.02s/it]

2022-12-09 11:51:05,250 Evaluating as a multi-label problem: False
2022-12-09 11:51:05,263 0.0	0.0	0.0	0.0
2022-12-09 11:51:05,264 
Results:
- F-score (micro) 0.0
- F-score (macro) 0.0
- Accuracy 0.0

By class:
              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000     345.0
         ORG     0.0000    0.0000    0.0000     112.0
         PER     0.0000    0.0000    0.0000     106.0
        MISC     0.0000    0.0000    0.0000      75.0
        TIME     0.0000    0.0000    0.0000      65.0

   micro avg     0.0000    0.0000    0.0000     703.0
   macro avg     0.0000    0.0000    0.0000     703.0
weighted avg     0.0000    0.0000    0.0000     703.0

2022-12-09 11:51:05,265 ----------------------------------------------------------------------------------------------------


{'test_score': 0.0,
 'dev_score_history': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'train_loss_history': [1.8710815052353171,
  0.934886026183842,
  0.880370293021754,
  0.8380933155376087,
  0.8005759801253864,
  0.781245861723047],
 'dev_loss_history': [0.9319246411323547,
  0.8952202200889587,
  0.8871385455131531,
  0.7917019724845886,
  0.8529025316238403,
  0.7337616086006165]}

# Stacked (Word + Character)

## On domain-specific (re3d) data

In [18]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='val.conll')

2022-12-12 11:26:11,206 Reading data from entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits
2022-12-12 11:26:11,207 Train: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/train.conll
2022-12-12 11:26:11,208 Dev: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/val.conll
2022-12-12 11:26:11,208 Test: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/test.conll


In [20]:
# Define tags to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2022-12-12 11:26:15,652 Computing label dictionary. Progress:


771it [00:00, 49790.73it/s]

2022-12-12 11:26:15,673 Dictionary created for label 'ner' with 11 values: Organisation (seen 1113 times), Location (seen 647 times), Person (seen 415 times), Temporal (seen 179 times), Quantity (seen 151 times), Weapon (seen 81 times), DocumentReference (seen 50 times), MilitaryPlatform (seen 48 times), Nationality (seen 34 times), Money (seen 20 times)


In [21]:
# Embeddings

from typing import List
from flair import embeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, CharacterEmbeddings, ELMoEmbeddings, ELMoTransformerEmbeddings, DocumentEmbeddings

In [22]:
embedding_types = [
    # init standard GloVe embedding
    WordEmbeddings('glove'),

    # init Flair forward and backwards embeddings
    CharacterEmbeddings()
]

embeddings = StackedEmbeddings(embeddings=embedding_types)


2022-12-12 11:26:26,839 https://flair.informatik.hu-berlin.de/resources/characters/common_characters not found in cache, downloading to /tmp/tmp22vpeed7


100%|██████████| 2887/2887 [00:00<00:00, 2234949.36B/s]

2022-12-12 11:26:27,966 copying /tmp/tmp22vpeed7 to cache at /home/digitalhub/.flair/datasets/common_characters
2022-12-12 11:26:27,967 removing temp file /tmp/tmp22vpeed7


In [23]:
from flair.models import SequenceTagger

# initialize a BiLSTM-CRF sequence tagger
tagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

from flair.trainers import ModelTrainer

# initialize trainer
trainer = ModelTrainer(tagger, corpus)

# start training
trainer.train('resources/taggers/re3d-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-12-12 11:26:29,714 SequenceTagger predicts: Dictionary with 41 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=

/home/digitalhub/.pyenv/versions/3.8.12/lib/python3.8/site-packages/flair/trainers/trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2022-12-12 11:26:30,617 epoch 1 - iter 2/25 - loss 3.21932688 - samples/sec: 72.98 - lr: 0.100000
2022-12-12 11:26:31,638 epoch 1 - iter 4/25 - loss 2.70893033 - samples/sec: 62.78 - lr: 0.100000
2022-12-12 11:26:32,660 epoch 1 - iter 6/25 - loss 2.40723167 - samples/sec: 62.74 - lr: 0.100000
2022-12-12 11:26:33,748 epoch 1 - iter 8/25 - loss 2.19683027 - samples/sec: 58.89 - lr: 0.100000
2022-12-12 11:26:34,642 epoch 1 - iter 10/25 - loss 2.09225777 - samples/sec: 71.65 - lr: 0.100000
2022-12-12 11:26:35,581 epoch 1 - iter 12/25 - loss 2.00498739 - samples/sec: 68.26 - lr: 0.100000
2022-12-12 11:26:38,066 epoch 1 - iter 14/25 - loss 1.99741056 - samples/sec: 25.77 - lr: 0.100000
2022-12-12 11:26:39,296 epoch 1 - iter 16/25 - loss 1.92919313 - samples/sec: 52.10 - lr: 0.100000
2022-12-12 11:26:40,217 epoch 1 - iter 18/25 - loss 1.90051307 - samples/sec: 69.58 - lr: 0.100000
2022-12-12 11:26:41,362 epoch 1 - iter 20/25 - loss 1.90873228 - samples/sec: 55.97 - lr: 0.100000
2022-12-12 11:

100%|██████████| 4/4 [00:00<00:00,  8.20it/s]

2022-12-12 11:26:44,284 Evaluating as a multi-label problem: False
2022-12-12 11:26:44,296 DEV : loss 1.450675368309021 - f1-score (micro avg)  0.0
2022-12-12 11:26:44,304 BAD EPOCHS (no improvement): 0
2022-12-12 11:26:44,305 ----------------------------------------------------------------------------------------------------


...

2022-12-12 11:49:01,936 Evaluating as a multi-label problem: False
2022-12-12 11:49:01,944 DEV : loss 0.2617446184158325 - f1-score (micro avg)  0.6588
2022-12-12 11:49:01,952 Epoch    93: reducing learning rate of group 0 to 9.7656e-05.
2022-12-12 11:49:01,954 BAD EPOCHS (no improvement): 4
2022-12-12 11:49:01,955 ----------------------------------------------------------------------------------------------------
2022-12-12 11:49:01,955 ----------------------------------------------------------------------------------------------------
2022-12-12 11:49:01,956 learning rate too small - quitting training!
2022-12-12 11:49:01,956 ----------------------------------------------------------------------------------------------------


2022-12-12 11:49:02,390 ----------------------------------------------------------------------------------------------------
2022-12-12 11:49:02,391 loading file resources/taggers/re3d-ner/best-model.pt
2022-12-12 11:49:02,640 SequenceTagger predicts: Dictionary with 43 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money, <START>, <STOP>


100%|██████████| 3/3 [00:00<00:00,  6.59it/s]

2022-12-12 11:49:03,136 Evaluating as a multi-label problem: False
2022-12-12 11:49:03,143 0.6523	0.6176	0.6345	0.4736
2022-12-12 11:49:03,143 
Results:
- F-score (micro) 0.6345
- F-score (macro) 0.3169
- Accuracy 0.4736

By class:
                   precision    recall  f1-score   support

     Organisation     0.6496    0.6357    0.6426       140
         Location     0.6061    0.5882    0.5970        68
           Person     0.7419    0.7419    0.7419        62
         Temporal     0.7619    0.8421    0.8000        19
         Quantity     0.4000    0.3750    0.3871        16
DocumentReference     0.0000    0.0000    0.0000         4
           Weapon     0.0000    0.0000    0.0000         4
 MilitaryPlatform     0.0000    0.0000    0.0000         3
      Nationality     0.0000    0.0000    0.0000         2
            Money     0.0000    0.0000    0.0000         1

        micro avg     0.6523    0.6176    0.6345       319
        macro avg     0.3160    0.3183    0.3169       319

{'test_score': 0.6344605475040257,
 'dev_score_history': [0.0,
  0.0,
  0.022922636103151862,
  0.07027027027027027,
  0.11165048543689318,
  0.20179372197309417,
  0.2570806100217865,
  0.3063829787234042,
  0.21078431372549017,
  0.35958904109589046,
  0.34146341463414637,
  0.4212523719165086,
  0.4725274725274725,
  0.39215686274509803,
  0.40408163265306124,
  0.5130718954248366,
  0.46307385229540915,
  0.5135135135135136,
  0.5103189493433395,
  0.543046357615894,
  0.56,
  0.5537974683544303,
  0.5530546623794211,
  0.49508196721311476,
  0.5604900459418071,
  0.5551948051948051,
  0.5736196319018404,
  0.5780730897009967,
  0.5321375186846039,
  0.5844155844155844,
  0.5388127853881278,
  0.5965463108320251,
  0.583196046128501,
  0.5976331360946746,
  0.6130952380952382,
  0.5993690851735015,
  0.5856697819314641,
  0.608955223880597,
  0.5945945945945946,
  0.613953488372093,
  0.6188340807174888,
  0.6174298375184638,
  0.624405705229794,
  0.6212361331220285,
  0.621417797

## On general (docred) data

In [18]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'docred_preprocessed'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

downsampled_corpus = corpus.downsample(0.1)

2022-12-09 11:16:58,545 Reading data from docred_preprocessed
2022-12-09 11:16:58,547 Train: docred_preprocessed/train.txt
2022-12-09 11:16:58,549 Dev: docred_preprocessed/dev.txt
2022-12-09 11:16:58,549 Test: docred_preprocessed/test.txt


In [22]:
print(downsampled_corpus.train[0].to_tagged_string('ner'))
print(len(downsampled_corpus.train))

Sentence: "Del ia ĠBuckley ĠGallagher Ġ( Ġborn Ġ11 ĠMarch Ġ1970 Ġ) Ġis Ġan ĠAmerican Ġjournalist Ġbased Ġin ĠRome Ġwho Ġcurrently Ġserves Ġas Ġthe ĠSenior ĠEditor Ġfor ĠInside Ġthe ĠVatican Ġmagazine Ġ. She Ġformerly Ġserved Ġas ĠCNN ĠâĢ Ļ s ĠFaith Ġand ĠValues ĠCorrespond ent Ġ. Based Ġin ĠNew ĠYork Ġ, ĠGallagher Ġwas Ġa Ġlong Ġtime ĠCNN ĠVatican ĠAnalyst Ġ, ĠVatican ologist Ġ, Ġand Ġreligious Ġjournalist Ġ. Prior Ġto Ġjoining ĠCNN Ġfull Ġtime Ġ, Ġshe Ġlived Ġin ĠRome Ġfor Ġ7 Ġyears Ġ. In ĠRome Ġ, Ġshe Ġwrote Ġa Ġweekly Ġcolumn Ġfor ĠZen it ĠNews ĠAgency Ġand Ġwas Ġa Ġcontributing Ġeditor Ġfor Ġthe Ġmagazine Ġ. The ĠHistory ĠChannel ĠDocument ary Ġ, Ġ" ĠAngels Ġand ĠDemons ĠDec oded Ġ" Ġreleased Ġby ĠA Ġ& ĠE ĠTelevision ĠNetworks Ġprof iled ĠGallagher Ġcommenting Ġon ĠDan ĠBrown Ġ' s Ġbest seller Ġbook Ġwhich Ġwas Ġmade Ġinto Ġa Ġmovie Ġ. Sub sequently Ġ, ĠGallagher Ġmoder ated Ġthe ĠAngels Ġand ĠDemons Ġmovie Ġpress Ġconference Ġfilm Ġdebut Ġin ĠRome Ġon Ġstage Ġwith ĠTom ĠH anks Ġ, 

In [9]:
# Define tags to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = downsampled_corpus.make_label_dictionary(label_type=tag_type)

2022-12-09 10:56:58,844 Computing label dictionary. Progress:


3008it [00:00, 25790.95it/s]

2022-12-09 10:56:59,000 Dictionary created for label 'ner' with 7 values: LOC (seen 10903 times), PER (seen 5336 times), MISC (seen 4419 times), ORG (seen 4334 times), TIME (seen 3340 times), NUM (seen 8 times)


In [10]:
# Embeddings

from typing import List
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

In [11]:
embedding_types = [
    # init standard GloVe embedding
    WordEmbeddings('glove'),

    # init Flair forward and backwards embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)


In [23]:
from flair.models import SequenceTagger

# initialize a BiLSTM-CRF sequence tagger
tagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

from flair.trainers import ModelTrainer

# initialize trainer
trainer = ModelTrainer(tagger, downsampled_corpus)

# start training
trainer.train('resources/taggers/docred-ner',
              learning_rate=0.1,
              mini_batch_size=64,
              max_epochs=150)

2022-12-09 11:18:12,896 SequenceTagger predicts: Dictionary with 25 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, S-TIME, B-TIME, E-TIME, I-TIME, S-NUM, B-NUM, E-NUM, I-NUM
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked

100%|██████████| 1/1 [00:29<00:00, 29.36s/it]

2022-12-09 11:24:50,576 Evaluating as a multi-label problem: False
2022-12-09 11:24:50,581 DEV : loss 0.9319246411323547 - f1-score (micro avg)  0.0
2022-12-09 11:24:50,600 BAD EPOCHS (no improvement): 0
2022-12-09 11:24:50,601 ----------------------------------------------------------------------------------------------------


2022-12-09 11:25:20,274 epoch 2 - iter 1/5 - loss 0.90134813 - samples/sec: 2.16 - lr: 0.100000
2022-12-09 11:25:48,091 epoch 2 - iter 2/5 - loss 0.93840038 - samples/sec: 2.30 - lr: 0.100000
2022-12-09 11:26:19,322 epoch 2 - iter 3/5 - loss 0.91924218 - samples/sec: 2.05 - lr: 0.100000
2022-12-09 11:26:45,252 epoch 2 - iter 4/5 - loss 0.93913768 - samples/sec: 2.47 - lr: 0.100000
2022-12-09 11:27:10,925 epoch 2 - iter 5/5 - loss 0.93488603 - samples/sec: 2.49 - lr: 0.100000
2022-12-09 11:27:10,926 ----------------------------------------------------------------------------------------------------
2022-12-09 11:27:10,926 EPOCH 2 done: loss 0.9349 - lr 0.100000


100%|██████████| 1/1 [00:01<00:00,  1.64s/it]

...

2022-12-09 11:49:36,713 Evaluating as a multi-label problem: False
2022-12-09 11:49:36,727 DEV : loss 0.7337616086006165 - f1-score (micro avg)  0.0
2022-12-09 11:49:36,749 BAD EPOCHS (no improvement): 0
2022-12-09 11:49:36,750 ----------------------------------------------------------------------------------------------------


2022-12-09 11:50:04,201 ----------------------------------------------------------------------------------------------------
2022-12-09 11:50:04,202 Exiting from training early.
2022-12-09 11:50:04,203 Saving model ...
2022-12-09 11:50:05,187 Done.
2022-12-09 11:50:05,188 ----------------------------------------------------------------------------------------------------
2022-12-09 11:50:05,188 Testing using last state of model ...


100%|██████████| 2/2 [01:00<00:00, 30.02s/it]

2022-12-09 11:51:05,250 Evaluating as a multi-label problem: False
2022-12-09 11:51:05,263 0.0	0.0	0.0	0.0
2022-12-09 11:51:05,264 
Results:
- F-score (micro) 0.0
- F-score (macro) 0.0
- Accuracy 0.0

By class:
              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000     345.0
         ORG     0.0000    0.0000    0.0000     112.0
         PER     0.0000    0.0000    0.0000     106.0
        MISC     0.0000    0.0000    0.0000      75.0
        TIME     0.0000    0.0000    0.0000      65.0

   micro avg     0.0000    0.0000    0.0000     703.0
   macro avg     0.0000    0.0000    0.0000     703.0
weighted avg     0.0000    0.0000    0.0000     703.0

2022-12-09 11:51:05,265 ----------------------------------------------------------------------------------------------------


{'test_score': 0.0,
 'dev_score_history': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'train_loss_history': [1.8710815052353171,
  0.934886026183842,
  0.880370293021754,
  0.8380933155376087,
  0.8005759801253864,
  0.781245861723047],
 'dev_loss_history': [0.9319246411323547,
  0.8952202200889587,
  0.8871385455131531,
  0.7917019724845886,
  0.8529025316238403,
  0.7337616086006165]}

# Transformer Word Embeddings

## On domain-specific (re3d) data

In [ ]:
from flair.embeddings import TransformerWordEmbeddings

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/sota-ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

2022-11-30 10:57:44,994 SequenceTagger predicts: Dictionary with 41 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money
2022-11-30 10:57:44,999 ----------------------------------------------------------------------------------------------------
2022-11-30 10:57:45,000 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 1024, padding_idx=1)
        (position_embe

100%|██████████| 25/25 [00:26<00:00,  1.06s/it]

2022-11-30 11:14:46,124 Evaluating as a multi-label problem: False
2022-11-30 11:14:46,130 DEV : loss 1.2281770706176758 - f1-score (micro avg)  0.0106
2022-11-30 11:14:46,132 BAD EPOCHS (no improvement): 4
2022-11-30 11:14:46,133 ----------------------------------------------------------------------------------------------------


2022-11-30 11:16:25,474 epoch 2 - iter 19/193 - loss 1.70634697 - samples/sec: 0.77 - lr: 0.000005
2022-11-30 11:18:08,092 epoch 2 - iter 38/193 - loss 1.60078184 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:19:53,055 epoch 2 - iter 57/193 - loss 1.52618754 - samples/sec: 0.72 - lr: 0.000005
2022-11-30 11:21:36,149 epoch 2 - iter 76/193 - loss 1.47910050 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:23:18,441 epoch 2 - iter 95/193 - loss 1.48425810 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:24:58,056 epoch 2 - iter 114/193 - loss 1.48248315 - samples/sec: 0.76 - lr: 0.000005
2022-11-30 11:26:40,738 epoch 2 - iter 133/193 - loss 1.47193090 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:28:24,931 epoch 2 - iter 152/193 - loss 1.45062939 - samples/sec: 0.73 - lr: 0.000005
2022-11-30 11:30:03,297 epoch 2 - iter 171/193 - loss 1.42916158 - samples/sec: 0.77 - lr: 0.000005
2022-11-30 11:31:45,988 epoch 2 - iter 190/193 - loss 1.41856586 - samples/sec: 0.74 - lr: 0.000004
2022-

100%|██████████| 25/25 [00:27<00:00,  1.09s/it]

...

2022-11-30 13:55:45,704 Evaluating as a multi-label problem: False
2022-11-30 13:55:45,711 DEV : loss 0.4879798889160156 - f1-score (micro avg)  0.6458
2022-11-30 13:55:45,713 BAD EPOCHS (no improvement): 4


2022-11-30 13:55:47,663 ----------------------------------------------------------------------------------------------------
2022-11-30 13:55:47,665 Testing using last state of model ...


100%|██████████| 24/24 [00:25<00:00,  1.07s/it]

2022-11-30 13:56:13,300 Evaluating as a multi-label problem: False
2022-11-30 13:56:13,306 0.6446	0.7618	0.6983	0.5473
2022-11-30 13:56:13,307 
Results:
- F-score (micro) 0.6983
- F-score (macro) 0.3655
- Accuracy 0.5473

By class:
                   precision    recall  f1-score   support

     Organisation     0.6606    0.7786    0.7148       140
         Location     0.5934    0.7941    0.6792        68
           Person     0.7568    0.9032    0.8235        62
         Temporal     0.7826    0.9474    0.8571        19
         Quantity     0.2778    0.3125    0.2941        16
DocumentReference     0.3333    0.2500    0.2857         4
           Weapon     0.0000    0.0000    0.0000         4
            Money     0.0000    0.0000    0.0000         1
 MilitaryPlatform     0.0000    0.0000    0.0000         3
      Nationality     0.0000    0.0000    0.0000         2

        micro avg     0.6446    0.7618    0.6983       319
        macro avg     0.3404    0.3986    0.3655       319

{'test_score': 0.6982758620689655,
 'dev_score_history': [0.010554089709762533,
  0.33587786259541985,
  0.5598845598845599,
  0.5923984272608126,
  0.6229508196721312,
  0.6420765027322404,
  0.6610169491525424,
  0.6418732782369146,
  0.6481994459833795,
  0.6457765667574932],
 'train_loss_history': [2.9750299324347216,
  1.4151145429336154,
  1.102379407128485,
  0.8985091482578567,
  0.7659444744710038,
  0.7034005997481789,
  0.6549618372841,
  0.6058066446442902,
  0.57429869816408,
  0.5604896890343632],
 'dev_loss_history': [1.2281770706176758,
  0.859157383441925,
  0.6455650925636292,
  0.5550165176391602,
  0.5053684711456299,
  0.4809141457080841,
  0.45765113830566406,
  0.4889335632324219,
  0.46722859144210815,
  0.4879798889160156]}

In [ ]:
# 7. run training with XLM parameters (20 epochs, small LR)
from torch.optim.lr_scheduler import OneCycleLR

trainer.train('resources/taggers/ner-english-ontonotes-large',
              learning_rate=5.0e-6,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=20,
              scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.,
              )

In [ ]:
input_sentence = 'My name is Eugene, I currently live in Singapore, I work for DSO.'
tagger: SequenceTagger = SequenceTagger.load("/content/model/conllpp/final-model.pt")
sentence: Sentence = Sentence(input_sentence)
tagger.predict(sentence)
print(sentence.to_tagged_string())

# Zero-shot

## Pre-trained model

In [30]:
from flair.models import TARSTagger
from flair.data import Sentence

# 1. Load zero-shot NER tagger
tars = TARSTagger.load('tars-ner')

# 2. Prepare some test sentences
sentences = [
            'My name is Clara, I live in Singapore, I work for DSTA and my monthly pay is S$3000.',
            'Missiles and other explosives or ammonium nitrate are dangerous.',
            'Ammonium nitrate was also used in the IRA\'s bombing of London\'s Canary Wharf in 1996.',
            'These have included such naval craft as the ship of the line, battleship, submarine, and aircraft carrier; aircraft such as the fighter, bomber, and helicopter; and ground vehicles such as the chariot and tank.',
            'George Washington went to Washington'
            ]


# 3. Define some classes of named entities
labels = ['Organisation' , 'Location' , 'Person' , 'Temporal' , 'Quantity' , 'Weapon', 'Document Reference', 'Military Platform' , 'Nationality' , 'Money'] 
tars.add_and_switch_to_new_task('task 1', labels, label_type='ner')

# 4. Predict for these classes and print results
for i in sentences:
    sentence = Sentence(i)
    tars.predict(sentence)
    print(sentence.to_tagged_string("ner"))

2022-12-12 12:14:09,599 loading file /home/digitalhub/.flair/models/tars-ner.pt
2022-12-12 12:14:17,812 SequenceTagger predicts: Dictionary with 5 tags: O, S-entity, B-entity, E-entity, I-entity
Sentence: "My name is Clara , I live in Singapore , I work for DSTA and my monthly pay is S $ 3000 ." → ["Clara"/Person, "Singapore"/Location, "DSTA"/Organisation, "S $ 3000"/Money]
Sentence: "Missiles and other explosives or ammonium nitrate are dangerous ."
Sentence: "Ammonium nitrate was also used in the IRA 's bombing of London 's Canary Wharf in 1996 ." → ["IRA"/Organisation, "London"/Location, "Canary Wharf"/Location]
Sentence: "These have included such naval craft as the ship of the line , battleship , submarine , and aircraft carrier ; aircraft such as the fighter , bomber , and helicopter ; and ground vehicles such as the chariot and tank ."
Sentence: "George Washington went to Washington" → ["George Washington"/Person, "Washington"/Location]


## Training TARS model

In [4]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.models import TARSTagger
from flair.trainers import ModelTrainer

# 1. define label names in natural language since some datasets come with cryptic set of labels
label_name_map = {'Organisation': 'Organisation', 
                    'Location': 'Location',
                    'Person': 'Person',
                    'Temporal': 'Temporal',
                    'Quantity': 'Quantity',
                    'Weapon': 'Weapon',
                    'DocumentReference': 'Document Reference',
                    'MilitaryPlatform': 'Military Platform',
                    'Nationality': 'Nationality',
                    'Money': 'Money'

                  }

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='val.conll')


# 3. what label do you want to predict?
label_type = 'ner'

# 4. make a label dictionary
label_dict = corpus.make_label_dictionary(label_type=label_type)

# 5. start from our existing TARS base model for English
tars = TARSTagger.load("tars-ner")

# 5a: alternatively, comment out previous line and comment in next line to train a new TARS model from scratch instead
# tars = TARSClassifier(embeddings="bert-base-uncased")

# 6. switch to a new task (TARS can do multiple tasks so you must define one)
tars.add_and_switch_to_new_task(task_name="ner",
                                label_dictionary=label_dict,
                                label_type=label_type,
                                )

# 7. initialize the text classifier trainer
trainer = ModelTrainer(tars, corpus)

# 8. start the training
trainer.train(base_path='resources/taggers/trec',  # path to store the model artifacts
              learning_rate=0.02,  # use very small learning rate
              mini_batch_size=32,
              mini_batch_chunk_size=4,  # optionally set this if transformer is too much for your machine
              max_epochs=150,  # terminate after 10 epochs
              )


NameError: name 'ColumnCorpus' is not defined

In [ ]:
flair.set_seed(123)

# load your corpus (example is for WNUT 2020, downsampled to 1% of data)
corpus = NER_ENGLISH_WNUT_2020().downsample(0.01)
dictionary = corpus.make_label_dictionary('ner')
print(dictionary)

# init the TARS sequence tagger
tars_tagger: FewshotClassifier = TARSTagger.load('tars-ner')

# add a new task
tars_tagger.add_and_switch_to_new_task(task_name="lab report NER",
                                       label_dictionary=dictionary,
                                       label_type="ner")

# train the model
trainer = ModelTrainer(tars_tagger, corpus, optimizer=torch.optim.AdamW)

trainer.train(
    f'resources/taggers/few-shot-sequence-tagger-wnut2020',
    learning_rate=5e-6,
    mini_batch_size=2, # set this higher if you have a big GPU
    max_epochs=10,
    embeddings_storage_mode="none",
    weight_decay=0.,
)